In [108]:
from typing import Dict, List, Type

import pandas as pd
import torch
from sklearn.decomposition import PCA
from torch import nn
from torch.nn import functional as F
from torchvision import transforms as T
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from torch.nn.functional import normalize as normalize_emb
from torchvision import models as tv_models

In [129]:
def preprocess_data(dataset):
    # Transformations
    resize = T.Resize((224, 224))
    normalize = T.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    )
    pil_to_image = T.PILToTensor()
    # Get models used to preprocess features
    mini_lm = SentenceTransformer("all-MiniLM-L6-v2")
    resnet = tv_models.resnet50(pretrained=True)
    resnet.eval()
    resnet.fc = nn.Identity()
    # Preprocess
    preprocessed = []
    for sample in tqdm(
        iterable=dataset,
        total=dataset.num_rows,
        desc="Processing data",
    ):
        image = sample["image"]
        label = sample["label"]
        # There are 4 images in "L" format
        if sample["image"].mode == "L":
            continue
        image = pil_to_image(image).float()
        resized_img = resize(image)
        normalized_img = normalize(resized_img)
        for description in sample["description"].split("\n"):
            if not description:
                continue
            with torch.no_grad():
                # Added batch dim
                img_emb = resnet(normalized_img.unsqueeze(dim=0))
                text_emb = mini_lm.encode(
                    sentences=description,
                    convert_to_tensor=True,
                )
            preprocessed.append(
                {
                    "img_emb": normalize_emb(img_emb[0], dim=0), # Drop batch dim
                    "text_emb": normalize_emb(text_emb, dim=0),
                    "image": resized_img,
                    "text": description,
                    "label": sample["label"],
                }
            )
    return pd.DataFrame(preprocessed)

## Preprocess Cube

In [127]:
dataset = load_dataset("alkzar90/CC6204-Hackaton-Cub-Dataset")

Found cached dataset cc6204-hackaton-cub-dataset (/home/cicheck/.cache/huggingface/datasets/alkzar90___cc6204-hackaton-cub-dataset/default/0.0.0/de850c9086bff0dd6d6eab90f79346241178f65e1a016a50eec240ae9cdf2064)


  0%|          | 0/2 [00:00<?, ?it/s]

In [130]:
preprocessed_train = preprocess_data(dataset["train"])

Processing data: 100%|████████████████████████████████████████████████████████████████| 5994/5994 [41:24<00:00,  2.41it/s]


In [131]:
preprocessed_train.head(2)

,img_emb,text_emb,image,text,label
0,"[tensor(0.0176), tensor(0.0005), tensor(0.0025...","[tensor(0.0832, device='cuda:0'), tensor(0.064...","[[[tensor(159.5414), tensor(161.8534), tensor(...",this bird is brown with a lighter brown crest.,0
1,"[tensor(0.0176), tensor(0.0005), tensor(0.0025...","[tensor(0.0553, device='cuda:0'), tensor(0.112...","[[[tensor(159.5414), tensor(161.8534), tensor(...","aquatic large bird with long hooked bill, whit...",0


In [132]:
print(preprocessed_train.shape)

(59900, 5)


In [133]:
preprocessed_train.to_pickle("data/cub/preprocessed_train.pkl")

In [135]:
preprocessed_test = preprocess_data(dataset["test"])

/home/cicheck/.pyenv/versions/3.10.2/envs/representation/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/cicheck/.pyenv/versions/3.10.2/envs/representation/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Processing data: 100%|████████████████████████████████████████████████████████████████| 5794/5794 [40:48<00:00,  2.37it/s]


In [137]:
preprocessed_test.to_pickle("data/cub/preprocessed_test.pkl")

In [ ]:
## Preprocess 